# Lecture 22
## Thursday, November 18th 2021
### Joins with `SQLite` and `pandas`

## Starting Up

You can connect to the saved database from last time if you want.
Alternatively, for extra practice, you can just recreate it from the datasets
provided in the `.txt` files.  That's what I'll do.

# Exercise Deliverables

> 1. Make a copy of this notebook and call it `L22_Exercises.ipynb`.
> 2. Do all exercises in _code cell(s) immediately after the "Exercise"
>    statement_ similarly to what you did in Lecture 21.
> 3. Be sure to upload your database with the
>    lecture exercises. You must name your database **`L22DB.sqlite`** and
>    **`L22DB_pandas.sqlite`**.
> 4. Commit your completed work to your class `git` repo inside the directory
>    `lectures/lecture22` on the `main` (or `master`) branch, similarly to what
>    you did in Lecture 21.


In [1]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L22DB.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        vals_to_insert = line.strip().split('|')
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors) # jump over the header
    for line in contributors.readlines():
        vals_to_insert = line.strip().split('|')[1:]
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

from IPython.display import display

## Recap

Last time, you played with a number of `SQLite` commands to query and update the
tables in the database.

One thing we didn't get to was how to query the contributors table based off of
a query in the candidates table.  For example, suppose you want to query which
contributors donated to Obama. You could use a nested `SELECT` statement to
accomplish that.

In [2]:
query = '''SELECT * FROM contributors WHERE candidate_id = (SELECT id from candidates WHERE last_name = "Obama")'''
viz_tables(contributor_cols, query)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,50.00,2007-07-30,20
1,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.00,2007-08-16,20
2,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.00,2007-09-19,20
3,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.00,2007-08-14,20
4,30,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,-2300.00,2007-08-14,20
5,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.00,2007-08-14,20
6,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.00,2007-09-25,20
7,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.00,2007-09-12,20
8,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,048431752,2300.00,2007-09-30,20
9,35,Buck,Barbara,,1780 NE 138th St,,North Miami,FL,331811316,50.00,2007-09-13,20


# Joins

The last example involved querying data from multiple tables.

In particular, we combined columns from the two related tables (related through
the `FOREIGN KEY`).

This leads to the idea of *joining* multiple tables together.  `SQL` has a set
of commands to handle different types of joins.  `SQLite` does not support the
full suite of join commands offered by `SQL` but you should still be able to get
the main ideas from the limited command set.

We'll begin with the `INNER JOIN`.

## INNER JOIN

The idea here is that you will combine the tables if the values of certain
columns are the same between the two tables.  In our example, we will join the
two tables based on the candidate `id`.  The result of the `INNER JOIN` will be
a new table consisting of the columns we requested and containing the common
data.  Since we are joining based off of the candidate `id`, we will not be
excluding any rows.

### Example

Here are two tables.  Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

Table A is associated with Table B through a foreign key on the id column.

If we join the two tables by comparing the id columns and selecting the nA, nB,
and attr columns then we'll get 

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |


The `SQLite` code to do this join would be 

```SQL
SELECT nA, A.attr, nB, B.attr FROM A INNER JOIN B ON B.idB = A.idA
```

Notice that the second row in table B is gone because the id values are not the
same.

### Visualization

What is `SQL` doing with this operation?  It may help to visualize this with a
Venn diagram.  Table A has rows with values corresponding to the `idA`
attribute.  Column B has rows with values corresponding to the `idB` attribute.
The `INNER JOIN` will combine the two tables such that rows with common entries
in the `id` attributes are included.  We essentially have the following Venn
diagram.

![]({static}../fig/inner_join.png)

## Exercises

1. Using an `INNER JOIN`, join the `candidates` and `contributors` tables by
   comparing the `id` column in the `candidates` table with the `candidate_id`
   column in the `contributors` table.  Display your joined table with the
   columns `contributors.last_name`, `contributors.first_name`, `amount` and
   `candidates.last_name`.
2. Do the same inner join as in the last part, but this time append a `WHERE`
   clause to select a specific candidate's last name.


In [3]:
col = ["contributors.last_name","contributors.first_name","amount", "candidates.last_name"]
query = '''SELECT contributors.last_name,contributors.first_name,amount, candidates.last_name FROM candidates INNER JOIN contributors ON contributors.candidate_id = candidates.id'''
viz_tables(col, query).head()
#cursor.execute(query).fetchall()

,contributors.last_name,contributors.first_name,amount,candidates.last_name
0,Agee,Steven,500.0,Huckabee
1,Ahrens,Don,250.0,Huckabee
2,Ahrens,Don,50.0,Huckabee
3,Ahrens,Don,100.0,Huckabee
4,Akin,Charles,100.0,Huckabee


In [4]:
query = '''SELECT contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name 
FROM candidates INNER JOIN contributors ON contributors.candidate_id = candidates.id WHERE
lower(candidates.last_name) == "obama" '''
viz_tables(["contributors.last_name","contributors.first_name","contributors.amount", "candidates.last_name"], query).head()
#cursor.execute(query).fetchall()

,contributors.last_name,contributors.first_name,contributors.amount,candidates.last_name
0,Buckler,Steve,50.0,Obama
1,Buckler,Steve,25.0,Obama
2,Buckheit,Bruce,100.0,Obama
3,Buckel,Linda,2300.0,Obama
4,Buckel,Linda,-2300.0,Obama


## `LEFT JOIN` or `LEFT OUTER JOIN`

There are many ways to combine two tables.  We just explored one possibility in
which we combined the tables based upon the intersection of the two tables (the
`INNER JOIN`).

Now we'll look at the `LEFT JOIN` or `LEFT OUTER JOIN` in some databases.

In words, the `LEFT JOIN` is combining the tables based upon what is in the
intersection of the two tables *and* what is in the "reference" table (left
table in the `SQL` command).

We can consider our toy example in two guises:

#### Example A

Let's do a `LEFT JOIN` of table B from table A.  That is, we'd like to make a
new table by putting table B into table A.  In this case, we'll consider table A
our "reference" table.  We're comparing by the `id` column again.  We know that
these two tables share ids 0 and 2 and table A doesn't have anything else in it.
The resulting table is:

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |

That's not very exciting.  It's the same result as from the `INNER JOIN`.  We
can do another example that may be more enlightening.

#### Example B

Let's do a `LEFT JOIN` of table A from table B.  That is, we'd like to make a
new table by putting table A into table B.  In this case, we'll consider table B
our "reference" table.  Again, we use the `id` column from comparison.  We know
that these two tables share ids 0 and 2.  This time, table B also contains the
id 7, which is not shared by table A.  The resulting table is:

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| None | NaN     | t2  | 14     |
| s2   | 7       | t3  | 22     |

Notice that `SQLite` filled in the missing entries for us.  This is necessary
for completion of the requested join.

The `SQLite` commands to accomplish all of this are:

```SQL
SELECT nA, A.attr, nB, B.attr FROM A LEFT JOIN B ON B.idB = A.idA
```

and

```SQL
SELECT nA, A.attr, nB, B.attr FROM B LEFT JOIN A ON A.idA = B.idB
```

Here is a visualization using Venn diagrams of the `LEFT JOIN`.

![]({static}../fig/left_join.png)

## Exercises

Use the following two tables to do the first two exercises in this section.
Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |
| s3   | 15    | 3   |
| s4   | 31    | 7   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

1. Write the markdown table that would result from a `LEFT JOIN` using table A
   as the reference and the `id` columns for comparison.  Example with arbitrary
   column names:
   ```md
   | Col1 | Col2 | Col3 | Col4 |
   | :::: | :::: | :::: | :::: |
   | val1 | val2 | val3 | val4 |
   | val5 | val6 | val7 | val8 |
   ```
2. Write the markdown table that would result from a `LEFT JOIN` using table B
   as the reference and the `id` columns for comparison.
3. Now back to the candidates and their contributors.  Create a new table with
   the following form:
   ```md
   | average contribution | candidate last name |
   | :::::::::::::::::::: | ::::::::::::::::::: |
   |    ...               |    ...              |
   ```
   The table should be created using the `LEFT JOIN` clause on the
   `contributors` table by joining the `candidates` table by the `id` column.
   The `average contribution` column should be obtained using the `AVG()` `SQL`
   function. Use the `GROUP BY` clause on the `candidates` last name.

---

Left JOIN using table A as the reference

|na|attr|idA|nB|attr|idb|
|::|::::|:::|::|::::|:::|
|s1|23. | 0 |t1|60. | 0 |
|s2|7.  | 2 |t3|22. |2. |
|s3| 15 |3. |null|null|null|
|s4|31. |7. |t2|14|7|

LEFT Join using Table B as the reference

|nB|attr|idb|na|attr|idA|
|::|::::|:::|::|::::|:::|
|t1|60. | 0 |s1|23. | 0 |
|t2|14|7|s4|31. |7.| 
|t3|22. |2. |s2|7.  | 2 |


  The table should be created using the `LEFT JOIN` clause on the
   `contributors` table by joining the `candidates` table by the `id` column.
   The `average contribution` column should be obtained using the `AVG()` `SQL`
   function. Use the `GROUP BY` clause on the `candidates` last name.

In [5]:
query = '''SELECT AVG(amount), candidates.last_name AS Amount_avg FROM contributors LEFT JOIN candidates 
ON contributors.candidate_id = candidates.id GROUP BY candidates.last_name'''
viz_tables(["Amount_avg","candidates.last_name "], query)

,Amount_avg,candidates.last_name
0,50.6000,Clinton
1,-1360.0000,Giuliani
2,772.0000,Huckabee
3,452.6000,McCain
4,472.4356,Obama
5,-519.7088,Paul
6,156.0000,Romney


# `pandas`

We've been working with databases for the last few lectures and learning
`SQLite` commands to work with and manipulate the `SQL` databases.  There is a
powerful `python` package called `pandas` that provides broad support for data
structures.  It can be used to interact with relational databases through its
own methods and even through `SQL` commands.

> In the last part of this lecture, you will get to redo a number of the
> previous database exercises using `pandas`.

We won't be able to cover `pandas` from the ground up, but it's a
well-documented library and is fairly easy to get up and running.  The website
can be found at the following link: [`pandas`](http://pandas.pydata.org/).  A
very good reference for `pandas` is the book ["Python for data
analysis"](https://www.amazon.com/gp/product/1491957662/ref=as_li_tl?ie=UTF8&tag=quantpytho-20&camp=1789&creative=9325&linkCode=as2&creativeASIN=1491957662&linkId=8c3bf87b221dbcd8f541f0db20d4da83)
by the creator of `pandas` himself.

## Reading a datafile into `pandas`

In [6]:
# Using pandas naming convention
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [7]:
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcontr.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


Reading things in is quite easy with `pandas`. Notice that `pandas` populates
empty fields with `NaN` values. The `id` column in the `contributors` dataset is
superfluous.  Let's delete it.

In [8]:
del dfcontr['id']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


Very nice!  And we used the `head` method to print out the first five rows.

## Creating a Table with `pandas`

We can use `pandas` to create tables in a database. First, let's create a new
(empty) `SQLite` database:

In [9]:
dbp = sqlite3.connect('L22DB_pandas.sqlite')
cursor = dbp.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

Last time, we opened the data files with `python` and then manually used
`SQLite` commands to populate the individual tables.  We can use `pandas`
instead like so:

In [10]:
dfcand.to_sql("candidates", dbp, if_exists="append", index=False)

What is the size of our table?

In [11]:
dfcand.shape

(17, 5)

We can visualize the data in our `pandas`-populated `SQL` table.  No surprises
here except that `pandas` did everything for us (contrast this to our manual
file read last time using `for`-loops and list comprehensions).

In [12]:
query = '''SELECT * FROM candidates'''
display(viz_tables(["Average Contribution", "candidates.last_name"], query))

,Average Contribution,candidates.last_name
0,16,Mike
1,20,Barack
2,22,Rudolph
3,24,Mike
4,26,John
5,29,Bill
6,30,Duncan
7,31,Dennis
8,32,Ron
9,33,Joseph


## Querying a table with `pandas`

### One Way

Using the `query` method:

In [13]:
dfcand.query("first_name=='Mike' & party=='D'")

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


### Another Way

Using the `__getitem__` special method:

In [14]:
dfcand[(dfcand.first_name=="Mike") & (dfcand.party=="D")]

,id,first_name,last_name,middle_name,party
6,24,Mike,Gravel,NaN,D


### More Queries

In [15]:
dfcand[dfcand.middle_name.notnull()]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
16,41,Fred,Thompson,D.,R


In [16]:
dfcand[dfcand.first_name.isin(['Mike', 'Hillary'])]

,id,first_name,last_name,middle_name,party
2,34,Hillary,Clinton,R.,D
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R


## Exercises

1. Use `pandas` to populate the `contributors` table and display the populated `SQL` table
2. Query the `contributors` tables with the following:
  1. List entries where the state is "VA" and the amount is less than
     $\$400.00$.
  2. List entries where the state is "NULL".
  3. List entries for the states of Texas and Pennsylvania.
  4. List entries where the amount contributed is between $\$10.00$ and
     $\$50.00$.

In [17]:
dfcontr.to_sql("contributors", dbp, if_exists="append", index=False)
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [18]:
dfcontr[(dfcontr.state == "VA")&(dfcontr.amount > 400.00)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16


In [19]:
dfcontr[(dfcontr.state ==None)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id


In [20]:
dfcontr[(dfcontr.state.isin(["TX","PA"]))]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
65,BURGERT,RONALD,L.,5723 PLUMTREE DRIVE,NaN,DALLAS,TX,752524926,-1000.00,2008-03-05,22
70,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
71,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
79,Muse,Nina,Jo,2915 Toro Canyon Rd,NaN,Austin,TX,78746,-50.00,2008-04-21,32
93,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.00,2008-04-21,32
94,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.00,2008-04-21,32
95,Johnson,Cathleen,E.,1003 Justin Ln Apt 2016,NaN,Austin,TX,787572648,-14.76,2008-04-21,32
132,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-09-30,35
133,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,150.00,2007-08-09,35
134,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-07-19,35


In [21]:
dfcontr[(dfcontr.amount < 50.00)&(dfcontr.amount >= 10.00)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
140,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
143,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
144,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
158,ABBO,PAULINE,MORENCY,10720 JACOB LANE,NaN,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
160,ABAIR,PETER,NaN,40 EVANS STREET,NaN,WATERTOWN,MA,24722150,25.0,2008-01-09,37


## Sorting

In [22]:
dfcand.sort_values(by='party')

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
13,29,Bill,Richardson,NaN,D
6,24,Mike,Gravel,NaN,D
11,20,Barack,Obama,NaN,D
9,31,Dennis,Kucinich,NaN,D
14,35,Mitt,Romney,NaN,R
12,32,Ron,Paul,NaN,R


In [23]:
dfcand.sort_values(by='party', ascending=False)

,id,first_name,last_name,middle_name,party
8,30,Duncan,Hunter,NaN,R
7,16,Mike,Huckabee,NaN,R
15,38,Tom,Tancredo,NaN,R
14,35,Mitt,Romney,NaN,R
12,32,Ron,Paul,NaN,R
10,37,John,McCain,NaN,R
1,36,Samuel,Brownback,NaN,R
16,41,Fred,Thompson,D.,R
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D


## Selecting Columns

In [24]:
dfcand[['last_name', 'party']]

,last_name,party
0,Biden,D
1,Brownback,R
2,Clinton,D
3,Dodd,D
4,Edwards,D
5,Giuliani,R
6,Gravel,D
7,Huckabee,R
8,Hunter,R
9,Kucinich,D


In [25]:
dfcand[['last_name', 'party']].count()

last_name    17
party        17
dtype: int64

In [26]:
dfcand[['first_name']].drop_duplicates()

,first_name
0,Joseph
1,Samuel
2,Hillary
3,Christopher
4,John
5,Rudolph
6,Mike
8,Duncan
9,Dennis
11,Barack


In [27]:
dfcand[['first_name']].drop_duplicates().count()

first_name    15
dtype: int64

## Exercises

1. Sort the contributors table by `amount` and order in *descending* order.
2. Select the `first_name` and `amount` columns.
3. Select the `last_name` and `first_name` columns and drop duplicates.
4. Count how many there are after the duplicates have been dropped.

In [28]:
dfcontr.sort_values(by='amount', ascending=False).head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20


In [29]:
dfcontr[["first_name", "amount"]].head()

,first_name,amount
0,Steven,500.0
1,Don,250.0
2,Don,50.0
3,Don,100.0
4,Charles,100.0


In [30]:
dfcontr[["first_name", "last_name"]].drop_duplicates().head()

,first_name,last_name
0,Steven,Agee
1,Don,Ahrens
4,Charles,Akin
5,Mike,Akin
6,Rebecca,Akin


In [31]:
dfcontr[["first_name", "last_name"]].drop_duplicates().count()

first_name    126
last_name     126
dtype: int64

## Altering Tables

Creating a new column is quite easy with `pandas`.

In [32]:
dfcand['name'] = dfcand['last_name'] + ", " + dfcand['first_name']
dfcand

,id,first_name,last_name,middle_name,party,name
0,33,Joseph,Biden,NaN,D,"Biden, Joseph"
1,36,Samuel,Brownback,NaN,R,"Brownback, Samuel"
2,34,Hillary,Clinton,R.,D,"Clinton, Hillary"
3,39,Christopher,Dodd,J.,D,"Dodd, Christopher"
4,26,John,Edwards,NaN,D,"Edwards, John"
5,22,Rudolph,Giuliani,NaN,R,"Giuliani, Rudolph"
6,24,Mike,Gravel,NaN,D,"Gravel, Mike"
7,16,Mike,Huckabee,NaN,R,"Huckabee, Mike"
8,30,Duncan,Hunter,NaN,R,"Hunter, Duncan"
9,31,Dennis,Kucinich,NaN,D,"Kucinich, Dennis"


We can change an existing field as well.

In [33]:
dfcand.loc[dfcand.first_name == "Mike", "name"]

6      Gravel, Mike
7    Huckabee, Mike
Name: name, dtype: object

In [34]:
dfcand.loc[dfcand.first_name == "Mike", "name"] = "Mikey"
dfcand

,id,first_name,last_name,middle_name,party,name
0,33,Joseph,Biden,NaN,D,"Biden, Joseph"
1,36,Samuel,Brownback,NaN,R,"Brownback, Samuel"
2,34,Hillary,Clinton,R.,D,"Clinton, Hillary"
3,39,Christopher,Dodd,J.,D,"Dodd, Christopher"
4,26,John,Edwards,NaN,D,"Edwards, John"
5,22,Rudolph,Giuliani,NaN,R,"Giuliani, Rudolph"
6,24,Mike,Gravel,NaN,D,Mikey
7,16,Mike,Huckabee,NaN,R,Mikey
8,30,Duncan,Hunter,NaN,R,"Hunter, Duncan"
9,31,Dennis,Kucinich,NaN,D,"Kucinich, Dennis"


In [35]:
dfcand.query("first_name == 'Mike'")
dfcand

,id,first_name,last_name,middle_name,party,name
0,33,Joseph,Biden,NaN,D,"Biden, Joseph"
1,36,Samuel,Brownback,NaN,R,"Brownback, Samuel"
2,34,Hillary,Clinton,R.,D,"Clinton, Hillary"
3,39,Christopher,Dodd,J.,D,"Dodd, Christopher"
4,26,John,Edwards,NaN,D,"Edwards, John"
5,22,Rudolph,Giuliani,NaN,R,"Giuliani, Rudolph"
6,24,Mike,Gravel,NaN,D,Mikey
7,16,Mike,Huckabee,NaN,R,Mikey
8,30,Duncan,Hunter,NaN,R,"Hunter, Duncan"
9,31,Dennis,Kucinich,NaN,D,"Kucinich, Dennis"


You may recall that `SQLite` doesn't have the functionality to drop a column.
It can be done with one line using `pandas`:

In [36]:
del dfcand['name']
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


## Exercises

1. Create a `name` column for the `contributors` table with field entries of the
   form "last name, first name"
2. For contributors from the state of "PA", change `name` to "X".
3. Delete the newly created name column.

In [37]:
dfcontr['name'] = dfcontr['last_name'] + ", " + dfcontr['first_name']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"


In [38]:
dfcontr.loc[dfcontr.state == "PA", "name"] = "X"
dfcontr.loc[dfcontr.state == "PA"]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
70,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22,X
71,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22,X
93,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.0,2008-04-21,32,X
94,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.0,2008-04-21,32,X
165,ABEL,JOHN,H.,422 THOMAS STREET,NaN,BETHLEHEM,PA,180153316,200.0,2008-01-22,37,X


In [39]:
del dfcontr['name']
dfcontr.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


## Aggregation

We'd like to get information about the tables such as the maximum amount
contributed to the candidates.  Basic statistics on a `pandas` frame can be
obtained using the `describe()` method:

In [40]:
dfcand.describe()

,id
count,17.000000
mean,30.764706
std,7.110990
min,16.000000
25%,26.000000
50%,32.000000
75%,36.000000
max,41.000000


It's not very interesting with the candidates table because the candidates table
only has one numeric column. Here are a few more data queries using the
contributors table:

In [41]:
dfcontr.amount.max()

4600.0

In [42]:
dfcontr[dfcontr.amount==dfcontr.amount.max()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20


In [43]:
dfcontr.groupby("state").sum()

,zip,amount,candidate_id
state,,,
AK,2985459621,1210.00,111
AR,864790,14200.00,192
AZ,860011121,120.00,37
CA,14736360720,-5013.73,600
CO,2405477834,-5823.00,111
CT,68901376,2300.00,35
DC,800341853,-1549.91,102
FL,8970626520,-4050.00,803
IA,50266,250.00,16


In [44]:
dfcontr.groupby("state")["amount"].sum()

state
AK     1210.00
AR    14200.00
AZ      120.00
CA    -5013.73
CO    -5823.00
CT     2300.00
DC    -1549.91
FL    -4050.00
IA      250.00
ID     -261.00
IL    -5586.80
KS     -330.00
KY     -200.00
LA     1300.00
MA      -83.00
MD      300.00
ME     2520.00
MI    -1265.00
MN      322.00
MO      100.00
NC      500.00
NH      -24.60
NJ     -817.45
NV      725.00
NY    -6474.50
OH      450.00
OK      800.00
PA    -2146.00
RI      200.00
SC     2400.00
TN      -25.00
TX     1985.24
UT     5050.00
VA      515.92
WA     -500.00
Name: amount, dtype: float64

In [45]:
dfcontr.state.unique()

array(['VA', 'CA', 'AR', 'DC', 'SC', 'IA', 'OH', 'NC', 'UT', 'MO', 'IL',
       'ME', 'FL', 'MD', 'MI', 'CO', 'WA', 'NY', 'TX', 'KY', 'PA', 'TN',
       'MA', 'MN', 'KS', 'NJ', 'NH', 'ID', 'OK', nan, 'NV', 'CT', 'RI',
       'AK', 'LA', 'AZ'], dtype=object)

There is also a version of the `LIMIT` clause in `SQL`.  It's very intuitive
using `pandas` in `python`:

In [46]:
dfcand[0:3]

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D


The usual `python` slicing works just fine!


## Exercise

Use the `describe()` method on the `contributors` table.

In [55]:
dfcontr.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000


## Joins with `pandas`

`pandas` has some some documentation on `joins`:  [Merge, join, and
concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html).  If you
want some more reinforcement on the concepts from earlier regarding `JOIN`, then
the `pandas` documentation may be a good place to get it.

You may also be interested in [a comparison with
`SQL`](http://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#compare-with-sql-join).

> To do joins with `pandas`, we use the `merge` method.

Here's an example of an explicit inner join:

In [47]:
cols_wanted = ['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y', 'amount']
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

,last_name_x,first_name_x,candidate_id,id,last_name_y,amount
0,Agee,Steven,16,16,Huckabee,500.00
1,Ahrens,Don,16,16,Huckabee,250.00
2,Ahrens,Don,16,16,Huckabee,50.00
3,Ahrens,Don,16,16,Huckabee,100.00
4,Akin,Charles,16,16,Huckabee,100.00
5,Akin,Mike,16,16,Huckabee,1500.00
6,Akin,Rebecca,16,16,Huckabee,500.00
7,Aldridge,Brittni,16,16,Huckabee,250.00
8,Allen,John D.,16,16,Huckabee,1000.00
9,Allen,John D.,16,16,Huckabee,1300.00


Somewhat more organized with additional grouping and description of resulting
data frame:

In [48]:
dfcontr.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted].groupby('last_name_y').describe()

candidate_id                                             id                                          amount                                                                  
                   count  mean  std   min   25%   50%   75%   max count  mean  std   min   25%   50%   75%   max  count       mean          std     min     25%     50%      75%      max
last_name_y                                                                                                                                                                              
Clinton             25.0  34.0  0.0  34.0  34.0  34.0  34.0  34.0  25.0  34.0  0.0  34.0  34.0  34.0  34.0  34.0   25.0    50.6000   538.764636 -2300.0    70.0   100.0   100.00  1000.00
Giuliani            25.0  22.0  0.0  22.0  22.0  22.0  22.0  22.0  25.0  22.0  0.0  22.0  22.0  22.0  22.0  22.0   25.0 -1360.0000   934.077085 -2300.0 -2300.0 -1000.0  -500.00  -100.00
Huckabee            25.0  16.0  0.0  16.0  16.0  16.0  16.0  16.0  25.0  16.0  0.0  16.0  16.0  16.0  16.0  16.0   25.0   772.0000   714.008637    50.0   250.0   500.0  1000.00  2300.00
McCain              25.0  37.0  0.0  37.0  37.0  37.0  37.0  37.0  25.0  37.0  0.0  37.0  37.0  37.0  37.0  37.0   25.0   452.6000   565.549364    25.0   100.0   200.0   500.00  2600.00
Obama               25.0  20.0  0.0  20.0  20.0  20.0  20.0  20.0  25.0  20.0  0.0  20.0  20.0  20.0  20.0  20.0   25.0   472.4356  1233.114920 -2300.0    50.0   200.0   500.00  4600.00
Paul                25.0  32.0  0.0  32.0  32.0  32.0  32.0  32.0  25.0  32.0  0.0  32.0  32.0  32.0  32.0  32.0   25.0  -519.7088   693.111403 -2592.0  -844.8  -223.0   -34.08   -14.76
Romney              25.0  35.0  0.0  35.0  35.0  35.0  35.0  35.0  25.0  35.0  0.0  35.0  35.0  35.0  35.0  35.0   25.0   156.0000   609.827503 -1000.0    50.0    75.0   250.00  2300.00

### Other Joins with `pandas`

We didn't cover all possible joins because `SQLite` can only handle the few that
we did discuss.  As mentioned, there are workarounds for some things in
`SQLite`, but not everything.  Fortunately, `pandas` can handle pretty much
everything.  Here are a few joins that `pandas` can handle:

* `LEFT OUTER`: discussed above
* `RIGHT OUTER`: think of the "opposite" of a `LEFT OUTER` join (shade the
  intersection and *right* set in the Venn diagram).
* `FULL OUTER`: combine everything from both tables (shade the entire Venn
  diagram)

Lets build the tables from the join exercise above in `pandas`:

In [49]:
df_A = pd.DataFrame({
    'nA': ['s1', 's2', 's3', 's4'],
    'attr': [23, 7, 15, 31],
    'idA': [0, 2, 3, 7]
})

df_B = pd.DataFrame({
    'nB': ['t1', 't2', 't3'],
    'attr': [60, 14, 22],
    'idB': [0, 7, 2]
})

#### Left Outer Join with `pandas`

In [50]:
df_A.merge(df_B, left_on='idA', right_on='idB', how='left')

,nA,attr_x,idA,nB,attr_y,idB
0,s1,23,0,t1,60.0,0.0
1,s2,7,2,t3,22.0,2.0
2,s3,15,3,NaN,NaN,NaN
3,s4,31,7,t2,14.0,7.0


#### Right Outer Join with `pandas`

In [51]:
df_A.merge(df_B, left_on='idA', right_on='idB', how='right')

,nA,attr_x,idA,nB,attr_y,idB
0,s1,23,0,t1,60,0
1,s4,31,7,t2,14,7
2,s2,7,2,t3,22,2


#### Full Outer Join with `pandas`

In [52]:
df_A.merge(df_B, left_on='idA', right_on='idB', how='outer')

,nA,attr_x,idA,nB,attr_y,idB
0,s1,23,0,t1,60.0,0.0
1,s2,7,2,t3,22.0,2.0
2,s3,15,3,NaN,NaN,NaN
3,s4,31,7,t2,14.0,7.0


# Save our databases

Commit the changes to the open `SQL` databases and close them.  Well done!

In [56]:
db.commit()
dbp.commit()
db.close()
dbp.close()